# **3182. Find Top Scoring Students**

**Table: students**
``` console
+-------------+----------+
| Column Name | Type     | 
+-------------+----------+
| student_id  | int      |
| name        | varchar  |
| major       | varchar  |
+-------------+----------+
``` 
student_id is the primary key (combination of columns with unique values) for this table.

Each row of this table contains the student ID, student name, and their major.
**Table: courses**
``` console
+-------------+----------+
| Column Name | Type     | 
+-------------+----------+
| course_id   | int      |
| name        | varchar  |
| credits     | int      |
| major       | varchar  |
+-------------+----------+
```
course_id is the primary key (combination of columns with unique values) for this table.

Each row of this table contains the course ID, course name, the number of credits for the course, and the major it belongs to.

**Table: enrollments**
``` console
+-------------+----------+
| Column Name | Type     | 
+-------------+----------+
| student_id  | int      |
| course_id   | int      |
| semester    | varchar  |
| grade       | varchar  |
+-------------+----------+
```
(student_id, course_id, semester) is the primary key (combination of columns with unique values) for this table.

Each row of this table contains the student ID, course ID, semester, and grade received.

Write a solution to find the students who have taken all courses offered in their major and have achieved a grade of A in all these courses.

Return the result table ordered by student_id in ascending order.

The result format is in the following example.

 

**Example:**

**Input:**

**students table:**
``` console
+------------+------------------+------------------+
| student_id | name             | major            |
+------------+------------------+------------------+
| 1          | Alice            | Computer Science |
| 2          | Bob              | Computer Science |
| 3          | Charlie          | Mathematics      |
| 4          | David            | Mathematics      |
+------------+------------------+------------------+
```
**courses table:**
``` console
+-----------+-----------------+---------+------------------+
| course_id | name            | credits | major            |
+-----------+-----------------+---------+------------------+
| 101       | Algorithms      | 3       | Computer Science |
| 102       | Data Structures | 3       | Computer Science |
| 103       | Calculus        | 4       | Mathematics      |
| 104       | Linear Algebra  | 4       | Mathematics      |
+-----------+-----------------+---------+------------------+
```
**enrollments table:**
``` console
+------------+-----------+----------+-------+
| student_id | course_id | semester | grade |
+------------+-----------+----------+-------+
| 1          | 101       | Fall 2023| A     |
| 1          | 102       | Fall 2023| A     |
| 2          | 101       | Fall 2023| B     |
| 2          | 102       | Fall 2023| A     |
| 3          | 103       | Fall 2023| A     |
| 3          | 104       | Fall 2023| A     |
| 4          | 103       | Fall 2023| A     |
| 4          | 104       | Fall 2023| B     |
+------------+-----------+----------+-------+
```
**Output:**
``` console
+------------+
| student_id |
+------------+
| 1          |
| 3          |
+------------+
```
**Explanation:**

Alice (student_id 1) is a Computer Science major and has taken both "Algorithms" and "Data Structures", receiving an 'A' in both.

Bob (student_id 2) is a Computer Science major but did not receive an 'A' in all required courses.

Charlie (student_id 3) is a Mathematics major and has taken both "Calculus" and "Linear Algebra", receiving an 'A' in both.

David (student_id 4) is a Mathematics major but did not receive an 'A' in all required courses.

Note: Output table is ordered by student_id in ascending order.

**Solution-1:**

``` sql
# Write your MySQL query statement below
with cte as(select a.student_id, 
count(b.course_id) over(partition by b.major,a.student_id) as major_cnt,
count(c.course_id) over(partition by a.major, c.student_id) as student_major_cnt,
SUM(case when grade = 'A' then 1 else 0 END) over(partition by a.major, c.student_id) as grade_cnt_in_major
from courses b 
LEFT JOIN students a
ON a.major = b.major
LEFT JOIN enrollments c
ON a.student_id = c.student_id and b.course_id = c.course_id)

select distinct student_id from cte
where student_major_cnt = major_cnt and major_cnt <= grade_cnt_in_major
order by student_id ASC

**Solution-2:**

``` sql
select a.student_id
from students a
left join courses b on a.major = b.major
left join enrollments c on a.student_id = c.student_id and b.course_id = c.course_id
group by 1
having count(distinct b.course_id) = count(distinct c.course_id)
and max(c.grade) = 'A'
order by 1

**Solution-3:**

``` sql
SELECT s.student_id
FROM students s
JOIN courses c
USING(major)
LEFT JOIN enrollments e
ON s.student_id = e.student_id AND c.course_id = e.course_id AND e.grade = 'A'
GROUP BY 1
HAVING COUNT(c.course_id) = SUM(e.grade = 'A') 
ORDER BY 1